In [63]:
from parallel_toktag import parallel_toktag
from urllib.parse import quote
from urllib import request
import codecs, jieba
import re, html, time
from os import listdir
from os.path import isfile, join
import random
from lmscore import *
toktagger = ZhTokTagger(
        tm=PyTablesTM('/pix/smttoktag/toktag.phrasetable.h5'),
        lm=KenLM('/pix/smttoktag/tag.blm'))
OPTION_LIST = ['a', 'b', 'c', 'd', 'e']
SAMPLE_PATH = '../pixbot/question_samples/'

SAMPLE_FILES = [f for f in listdir(path) if isfile(join(SAMPLE_PATH, f))]
def simple_preprocess(sample):
    no, content, a, b, c, d, e = re.findall(r'\[(\d+)\](.*)### a:(.*), b:(.*), c:(.*), d:(.*), e:(.*)\[END\]', sample)[0]
    options = [a.strip(), b.strip(), c.strip(), d.strip(), e.strip()]
    content = re.sub('(")(.?)', r'\2', content.strip())
    content = content.strip().replace('︽⊙＿⊙︽', '龐傍燮謝')
    wlist = list(jieba.cut(content))
    qidx = []
    i = 0
    for w in wlist:
        if w == '龐傍燮謝':
            wlist[i] = '*'
            qidx.append(i)
        i += 1
    return (no, wlist, qidx, options)

def get_sample(file_name, no_sample):
    with codecs.open(file_name, 'r', 'utf-8') as f:
        for line in f:
            for (no, content, a, b, c, d, e) in re.findall(r'\[(\d+)\](.*)### a:(.*), b:(.*), c:(.*), d:(.*), e:(.*)\[END\]', line):
                if int(no) == int(no_sample):
                    return line

def sample_lmscore(sample):
    no, content, a, b, c, d, e = re.findall(r'\[(\d+)\](.*)### a:(.*), b:(.*), c:(.*), d:(.*), e:(.*)\[END\]', sample)[0]
    content = re.sub('(")(.?)', r'\2', content.strip())
    sents = gen_answer_sentences(content, (a,b,c,d,e))
    seginfos = list(map(toktagger, sents))
    scores = list(map(lambda seginfo: zhseg_lm(seginfo.zh_seg), seginfos))
    return (no, OPTION_LIST[scores.index(max(scores))])

In [68]:
correct = 0
total = 0
for dummy in range(20):
    file = random.choice(SAMPLE_FILES)
    ans = []
    pred = ['-1', '-1', '-1', '-1', '-1','-1', '-1', '-1', '-1', '-1']
    with codecs.open(SAMPLE_PATH + file, 'r', 'utf-8') as f:
        for line in f:
            if line.find('[END]') < 0:
                ans = line.strip().split(' ')
            else:
                (no, a) = sample_lmscore(line.strip())
                pred[int(no)-1] = a
        for j in range(len(ans)):
            if pred[j] == ans[j]:
                correct += 1
            total += 1
        print(float(correct)/float(total))
print(float(correct)/float(total))
                

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.972972972972973
0.9761904761904762
0.9574468085106383
0.9615384615384616
0.9473684210526315
0.9516129032258065
0.9552238805970149
0.9583333333333334
0.961038961038961
0.9634146341463414
0.9540229885057471
0.9456521739130435
0.9456521739130435


In [69]:
total

92

In [71]:
print('asd: ',total)

asd:  92


0